In [95]:
import pandas as pd

df = pd.read_csv('../data/pmc_patients/PMC-Patients-oa-9995.csv')
print(df.head())
df.head(20)

   index patient_uid      PMID                         file_path  \
0      0   7665777-1  33492400  comm/PMC007xxxxxx/PMC7665777.xml   
1      1   7665777-2  33492400  comm/PMC007xxxxxx/PMC7665777.xml   
2      2   7665777-3  33492400  comm/PMC007xxxxxx/PMC7665777.xml   
3      3   7665777-4  33492400  comm/PMC007xxxxxx/PMC7665777.xml   
4      4   7665777-5  33492400  comm/PMC007xxxxxx/PMC7665777.xml   

                                               title  \
0  Early Physical Therapist Interventions for Pat...   
1  Early Physical Therapist Interventions for Pat...   
2  Early Physical Therapist Interventions for Pat...   
3  Early Physical Therapist Interventions for Pat...   
4  Early Physical Therapist Interventions for Pat...   

                                             patient               age gender  \
0  This 60-year-old male was hospitalized due to ...  [[60.0, 'year']]      M   
1  A 39-year-old man was hospitalized due to an i...  [[39.0, 'year']]      M   
2  One week

,index,patient_uid,PMID,file_path,title,patient,age,gender,relevant_articles,similar_patients
0,0,7665777-1,33492400,comm/PMC007xxxxxx/PMC7665777.xml,Early Physical Therapist Interventions for Pat...,This 60-year-old male was hospitalized due to ...,"[[60.0, 'year']]",M,"{'32320506': 1, '32293716': 1, '23219649': 1, ...","{'7665777-2': 2, '7665777-3': 2, '7665777-4': ..."
1,1,7665777-2,33492400,comm/PMC007xxxxxx/PMC7665777.xml,Early Physical Therapist Interventions for Pat...,A 39-year-old man was hospitalized due to an i...,"[[39.0, 'year']]",M,"{'32320506': 1, '32293716': 1, '23219649': 1, ...","{'7665777-1': 2, '7665777-3': 2, '7665777-4': ..."
2,2,7665777-3,33492400,comm/PMC007xxxxxx/PMC7665777.xml,Early Physical Therapist Interventions for Pat...,One week after a positive COVID-19 result this...,"[[57.0, 'year']]",M,"{'32320506': 1, '32293716': 1, '23219649': 1, ...","{'7665777-1': 2, '7665777-2': 2, '7665777-4': ..."
3,3,7665777-4,33492400,comm/PMC007xxxxxx/PMC7665777.xml,Early Physical Therapist Interventions for Pat...,This 69-year-old male was admitted to the ICU ...,"[[69.0, 'year']]",M,"{'32320506': 1, '32293716': 1, '23219649': 1, ...","{'7665777-1': 2, '7665777-2': 2, '7665777-3': ..."
4,4,7665777-5,33492400,comm/PMC007xxxxxx/PMC7665777.xml,Early Physical Therapist Interventions for Pat...,This 57-year-old male was admitted to the ICU ...,"[[57.0, 'year']]",M,"{'32320506': 1, '32293716': 1, '23219649': 1, ...","{'7665777-1': 2, '7665777-2': 2, '7665777-3': ..."
5,5,7665777-6,33492400,comm/PMC007xxxxxx/PMC7665777.xml,Early Physical Therapist Interventions for Pat...,This 52-year-old male tested COVID-19 positive...,"[[52.0, 'year']]",M,"{'32320506': 1, '32293716': 1, '23219649': 1, ...","{'7665777-1': 2, '7665777-2': 2, '7665777-3': ..."
6,6,7665777-7,33492400,comm/PMC007xxxxxx/PMC7665777.xml,Early Physical Therapist Interventions for Pat...,Paramedics found this 59-year-old female with ...,"[[59.0, 'year']]",F,"{'32320506': 1, '32293716': 1, '23219649': 1, ...","{'7665777-1': 2, '7665777-2': 2, '7665777-3': ..."
7,7,7665777-8,33492400,comm/PMC007xxxxxx/PMC7665777.xml,Early Physical Therapist Interventions for Pat...,This 33-year-old female patient had typical CO...,"[[33.0, 'year']]",F,"{'32320506': 1, '32293716': 1, '23219649': 1, ...","{'7665777-1': 2, '7665777-2': 2, '7665777-3': ..."
8,8,7665777-9,33492400,comm/PMC007xxxxxx/PMC7665777.xml,Early Physical Therapist Interventions for Pat...,This 66-year-old male patient was admitted to ...,"[[66.0, 'year']]",M,"{'32320506': 1, '32293716': 1, '23219649': 1, ...","{'7665777-1': 2, '7665777-2': 2, '7665777-3': ..."
9,9,7665777-10,33492400,comm/PMC007xxxxxx/PMC7665777.xml,Early Physical Therapist Interventions for Pat...,A 66-year-old male started to present symptoms...,"[[66.0, 'year']]",M,"{'32320506': 1, '32293716': 1, '23219649': 1, ...","{'7665777-1': 2, '7665777-2': 2, '7665777-3': ..."


In [98]:
# get patients with no similar patients
def get_unique_patients(_df):
    _df = _df.copy()
    return _df[_df['similar_patients'].apply(lambda x: len(x) == 2)]

no_similar_patients = get_unique_patients(df)
no_similar_patients.drop(columns=['similar_patients'], inplace=True)
no_similar_patients

,index,patient_uid,PMID,file_path,title,patient,age,gender,relevant_articles
13,13,8674405-1,34956746,comm/PMC008xxxxxx/PMC8674405.xml,Goserelin Ovarian Ablation Failure in Premenop...,A 36-year-old G4P2 premenopausal woman with a ...,"[[36.0, 'year']]",F,"{'29510273': 1, '10953134': 1, '30258937': 1, ..."
14,14,8674458-1,34956749,comm/PMC008xxxxxx/PMC8674458.xml,Treatment of Inferior Vena Cava Thrombosis by ...,The patient was a 62-year-old male with a past...,"[[62.0, 'year']]",M,"{'29396156': 1, '185006': 1, '15696057': 1, '2..."
15,15,8674685-1,34956752,comm/PMC008xxxxxx/PMC8674685.xml,Unusual Cutaneous Location of Langheransian Hi...,"We report a case of a 45-year-old woman, a non...","[[45.0, 'year']]",F,"{'30281871': 1, '9611381': 1, '26966089': 1, '..."
19,19,8675574-1,34956758,comm/PMC008xxxxxx/PMC8675574.xml,An Approach for Diagnosing and Treating Neuros...,A 71-year-old African American female with a m...,"[[71.0, 'year']]",F,"{'31577877': 1, '15477572': 1, '25387188': 1, ..."
20,20,8675577-1,34956771,comm/PMC008xxxxxx/PMC8675577.xml,An Extremely Rare Case of Metastatic Merkel Ca...,Our patient is a 78-year-old male with a past ...,"[[78.0, 'year']]",M,"{'25765179': 1, '32324435': 1, '16896047': 1, ..."
...,...,...,...,...,...,...,...,...,...
9973,9980,6623996-1,31312556,comm/PMC006xxxxxx/PMC6623996.xml,"Prostate Cancer, Gender Identity, and Testoste...","A 58-year-old male with KS, diagnosed at age 1...","[[58.0, 'year']]",M,"{'17183557': 1, '9645824': 1, '12050481': 1, '..."
9976,9983,6624119-1,31328075,comm/PMC006xxxxxx/PMC6624119.xml,Transesophageal Echocardiography During Cytore...,"The patient is a 69-year-old, 55 kg, 153 cm fe...","[[69.0, 'year']]",F,"{'30747965': 1, '24119169': 1, '20553586': 1, ..."
9977,9984,6624151-1,31312570,comm/PMC006xxxxxx/PMC6624151.xml,Submassive Pulmonary Embolism: A Re-evaluation...,A 55-year-old man with a history of type II di...,"[[55.0, 'year']]",M,"{'25433511': 1, '29248101': 1, '28460762': 1, ..."
9986,9993,6624460-1,31245979,comm/PMC006xxxxxx/PMC6624460.xml,A Rare Complication of Oropharyngeal Tularemia...,A 33-year-old man presented to our clinic with...,"[[33.0, 'year']]",M,"{'10879600': 1, '23836859': 1, '3892222': 1, '..."
